In [10]:
import pandas as pd

#Dataframes for each CSV File

df_agencies= pd.read_csv("agencies.csv") 
df_act_type = pd.read_csv("NIBRS_CRIMINAL_ACT_TYPE.csv")
df_act = pd.read_csv("NIBRS_CRIMINAL_ACT.csv")
df_incident = pd.read_csv("NIBRS_incident.csv")
df_location = pd.read_csv("NIBRS_LOCATION_TYPE.csv")
df_month = pd.read_csv("NIBRS_month.csv")
df_offense_type = pd.read_csv("NIBRS_OFFENSE_TYPE.csv")
df_offense = pd.read_csv("NIBRS_OFFENSE.csv")

df = df_incident\
    .merge(df_agencies, on="agency_id", how="outer")\
    .merge(df_month, on=["agency_id", "nibrs_month_id", "did"], how="outer")\
    .merge(df_offense, on="incident_id", how="outer", suffixes=("_left", "_right"))\
    .merge(df_offense_type, on="offense_code", how="outer")\
    .merge(df_act, on="offense_id", how="outer")\
    .merge(df_act_type, on="criminal_act_id", how="outer")\
    .merge(df_location, on="location_id", how = "outer")

df.to_csv("2022_merged_data.csv", index=False)

##### Removing Duplicates (If Any)

In [11]:
column_names = df.columns.tolist()
print("Number of Rows before removing duplicates: ", len(df))
df.drop_duplicates(subset=column_names, keep='first', inplace=True)
print('Number of Rows after removing Duplicates:', len(df))

Number of Rows before removing duplicates:  126788
Number of Rows after removing Duplicates: 126788


##### Removing Columns with Nulls or No Purpose

In [12]:
test = df.isna().sum()
for col, count in test.items():
    if count != 0:
        print(col, count)

data_year_x 39
agency_id 36
incident_id 39
nibrs_month_id 39
cargo_theft_flag 39
submission_date 39
incident_date 39
report_date_flag 39
incident_hour 39
cleared_except_id 39
cleared_except_date 126670
incident_status 39
data_home_x 126788
orig_format_x 39
did 39
yearly_agency_id 36
data_year_y 36
ori 36
legacy_ori 36
covered_by_legacy_ori 126788
direct_contributor_flag 36
dormant_flag 36
dormant_year 126788
reporting_type 36
ucr_agency_name 36
ncic_agency_name 482
pub_agency_name 36
pub_agency_unit 126533
agency_status 36
state_id 36
state_name 36
state_abbr 36
state_postal_abbr 36
division_code 36
division_name 36
region_code 36
region_name 36
region_desc 36
agency_type_name 36
population 36
submitting_agency_id 36
sai 36
submitting_agency_name 36
suburban_area_flag 36
population_group_id 36
population_group_code 36
population_group_desc 36
parent_pop_group_code 36
parent_pop_group_desc 36
mip_flag 36
pop_sort_order 36
summary_rape_def 126788
pe_reported_flag 36
male_officer 349
male

In [13]:
#These Columns had Null Entries
df.drop(columns=["cleared_except_date", 'data_home_x', 'covered_by_legacy_ori', 'dormant_year',\
                 'pub_agency_unit', 'report_date', 'update_flag', 'data_home_y', 'month_pub_status',\
                 'num_premises_entered', 'summary_rape_def', 'method_entry_code'], inplace=True)

#Manually Inspected The CSV
df.drop(columns=['nibrs_month_id_x', 'cargo_theft_flag', 'submission_date', 'report_date_flag', 'incident_id',\
                 'cleared_except_id', 'incident_status', 'orig_format_x', 'did', 'data_year_y', 'legacy_ori',\
                 'direct_contributor_flag', 'dormant_flag', 'reporting_type', 'ucr_agency_name', 'nibrs_month_id_x',\
                 'agency_status', 'state_id', 'state_abbr', 'state_postal_abbr', 'division_code', 'division_name',\
                 'region_code', 'region_name', 'region_desc', 'agency_type_name', 'submitting_agency_id', 'sai', 'ori',\
                 'submitting_agency_name', 'suburban_area_flag', 'population_group_id', 'population_group_code', 'yearly_agency_id',\
                 'population_group_desc', 'parent_pop_group_code', 'parent_pop_group_desc', 'mip_flag', 'pop_sort_order',\
                 'pe_reported_flag', 'officer_rate', 'employee_rate', 'nibrs_cert_date', 'nibrs_start_date', 'nibrs_leoka_start_date',\
                 'nibrs_ct_start_date',	'nibrs_multi_bias_start_date', 'nibrs_off_eth_start_date', 'covered_flag', 'county_name',\
                 'msa_name', 'publishable_flag', 'participated', 'nibrs_participated', 'data_year_left', 'nibrs_month_id_y',\
                 'agency_id_y',	'month_num', 'inc_data_year', 'reported_status', 'orig_format_y', 'ddocname', 'data_year_right',\
                 'offense_id', 'offense_code', 'attempt_complete_flag', 'location_id', 'ct_flag', 'hc_flag', 'hc_code', 'crime_against', \
                 'offense_group', 'data_year', 'criminal_act_id', 'criminal_act_code', 'criminal_act_desc', 'ncic_agency_name', 'location_code'],
                 inplace = True, errors='ignore')

df.rename(columns={'data_year_x': 'year'}, inplace=True)
df.rename(columns={'agency_id_x': 'agency_location'}, inplace=True)
df.rename(columns={'pub_agency_name': 'agency_name'}, inplace=True)
df.rename(columns={'state_name': 'state'}, inplace=True)
df.rename(columns={'location_name': 'location_area'}, inplace=True)

#ncic_agency_name


##### Check Datatype for each Variable. Change Date Datatype.

In [14]:
df.info()
df['incident_date'] = pd.to_datetime(df['incident_date'], format='%Y-%m-%d', errors='coerce')
print("\n")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126788 entries, 0 to 126787
Data columns (total 18 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   year                            126749 non-null  float64
 1   agency_id                       126752 non-null  float64
 2   nibrs_month_id                  126749 non-null  float64
 3   incident_date                   126749 non-null  object 
 4   incident_hour                   126749 non-null  float64
 5   agency_name                     126752 non-null  object 
 6   state                           126752 non-null  object 
 7   population                      126752 non-null  float64
 8   male_officer                    126439 non-null  float64
 9   male_civilian                   126439 non-null  float64
 10  male_officer+male_civilian      126439 non-null  float64
 11  female_officer                  126439 non-null  float64
 12  female_civilian 

In [ ]:
df.drop(columns=['nibrs_month_id', 'nibrs_month_id', 'state', 'criminal_act_name'], inplace=True)
df.dropna(inplace=True)
df['population'] = df['population'].astype(int)
df['year'] = df['year'].astype(int)
df['incident_hour'] = df['incident_hour'].astype(int)



In [16]:
df.to_csv("2022_merged_data.csv", index=False)

In [17]:
df.isna().sum()


year                              0
agency_id                         0
incident_date                     0
incident_hour                     0
agency_name                       0
population                        0
male_officer                      0
male_civilian                     0
male_officer+male_civilian        0
female_officer                    0
female_civilian                   0
female_officer+female_civilian    0
offense_name                      0
offense_category_name             0
criminal_act_name                 0
location_area                     0
dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17987 entries, 0 to 120567
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   year                            17987 non-null  int64         
 1   agency_id                       17987 non-null  float64       
 2   incident_date                   17987 non-null  datetime64[ns]
 3   incident_hour                   17987 non-null  int64         
 4   agency_name                     17987 non-null  object        
 5   population                      17987 non-null  int64         
 6   male_officer                    17987 non-null  float64       
 7   male_civilian                   17987 non-null  float64       
 8   male_officer+male_civilian      17987 non-null  float64       
 9   female_officer                  17987 non-null  float64       
 10  female_civilian                 17987 non-null  float64       
 11  female